<pre>
Table: Visits

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| visit_id    | int     |
| customer_id | int     |
+-------------+---------+
visit_id is the column with unique values for this table.
This table contains information about the customers who visited the mall.
 

Table: Transactions

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| transaction_id | int     |
| visit_id       | int     |
| amount         | int     |
+----------------+---------+
transaction_id is column with unique values for this table.
This table contains information about the transactions made during the visit_id.
 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

The result format is in the following example.

 

Example 1:

Input: 
Visits
+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+
Transactions
+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+
Output: 
+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+
Explanation: 
Customer with id = 23 visited the mall once and made one transaction during the visit with id = 12.
Customer with id = 9 visited the mall once and made one transaction during the visit with id = 13.
Customer with id = 30 visited the mall once and did not make any transactions.
Customer with id = 54 visited the mall three times. During 2 visits they did not make any transactions, and during one visit they made 3 transactions.
Customer with id = 96 visited the mall once and did not make any transactions.
As we can see, users with IDs 30 and 96 visited the mall one time without making any transactions. Also, user 54 visited the mall twice and did not make any transactions.
</pre>

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# importing pyspark sql functions
from pyspark.sql.functions import *

# importing sql types from pyspark
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType, DateType

# importing SparkSession
from pyspark.sql import SparkSession


In [0]:
# creating spark session and providing app name
spark = SparkSession.builder.appName("leetcode-top-50-sql-solution-with-pyspark").getOrCreate()

In [0]:
# creating Schema
# Define the schema for the Visits table
visits_schema = StructType([
    StructField("visit_id", IntegerType(), nullable=False),
    StructField("customer_id", IntegerType(), nullable=False)
])

# Define the schema for the Transactions table
transactions_schema = StructType([
    StructField("transaction_id", IntegerType(), nullable=False),
    StructField("visit_id", IntegerType(), nullable=False),
    StructField("amount", IntegerType(), nullable=False)
])



In [0]:

visits_df = spark.createDataFrame([
    (1, 23),
    (2, 9),
    (4, 30),
    (5, 54),
    (6, 96),
    (7, 54),
    (8, 54)
], schema=visits_schema)

transactions_df = spark.createDataFrame([
    (2, 5, 310),
    (3, 5, 300),
    (9, 5, 200),
    (12, 1, 910),
    (13, 2, 970)
], schema=transactions_schema)




In [0]:
visits_df.display()

visit_id,customer_id
1,23
2,9
4,30
5,54
6,96
7,54
8,54


In [0]:
transactions_df.display()

transaction_id,visit_id,amount
2,5,310
3,5,300
9,5,200
12,1,910
13,2,970


In [0]:
# Leetcode Solution in Spark SQL
# Creating Temporary view for the product dataframe for sql queries
visits_df.createOrReplaceTempView('visits')
transactions_df.createOrReplaceTempView('transactions')
sql_result = spark.sql(
    '''
    SELECT
    v.customer_id,
    count(v.visit_id) as count_no_trans
    FROM visits as v
    LEFT JOIN transactions as t
    ON v.visit_id = t.visit_id
    WHERE t.transaction_id IS NULL
    GROUP BY v.customer_id
    
    '''
)

# Displaying Result
sql_result.display()

customer_id,count_no_trans
30,1
96,1
54,2


In [0]:
# Leet Code Solution in Data Frame
new_transactions_df = transactions_df.withColumnRenamed('visit_id','tran_visit_id')
filter_result = visits_df.join(new_transactions_df,visits_df.visit_id == new_transactions_df.tran_visit_id,"left").filter(col('tran_visit_id').isNull())
# Displaying the filtered Result
final_result = filter_result.select('customer_id','visit_id').groupBy('customer_id').count()
final_result.withColumnRenamed('count','count_no_trans').display()

customer_id,count_no_trans
30,1
96,1
54,2
